In [1]:
from os import getcwd
from os.path import join, abspath, pardir
from json import dump
import geopandas as gp
from shapely.geometry import Point, LineString

##### Configs

In [7]:
parent_dir = abspath(join(getcwd(), pardir))
shape_files_dir = join(parent_dir, "2109_STIB_MIVB_Network")
line_shape_file = join(shape_files_dir, "ACTU_LINES.shp")
stops_shape_file = join(shape_files_dir, "ACTU_STOPS.shp")
output_file = join(parent_dir, "distances.json")

##### Load files

In [3]:
line_df = gp.read_file(line_shape_file)
stops_df = gp.read_file(stops_shape_file)

##### Method to calculate the distance between two stops for a given transport route

In [4]:
def calculate_distance_between_stops(stops_data, points):
    if not isinstance(stops_data, dict): raise ValueError("Kindly, pass the stops data with 'stop_id' and 'stop_points'.")
    if not isinstance(points, LineString): raise ValueError("Kindly, pass the correct points data.")
    previous_stop_id, previous_distance, distance = None, None, list()
    for index, stop_id in enumerate(stops_data):
        stop_point = stops_data[stop_id]
        stop_distance = points.project(stop_point)
        if index != 0:
            v = {"from": previous_stop_id, "to": stop_id, "distance": float(stop_distance - previous_distance) }
            distance.append(v)
        previous_stop_id = stop_id
        previous_distance = stop_distance
    return distance

##### Get Distance between stops for all the variants for a given transport

In [5]:
def get_stop_distances_for_transport(transport, line_df, stops_df):
    lines = line_df.loc[line_df['LIGNE'].isin([transport])]
    stops = stops_df.loc[stops_df['Code_Ligne'].isin([transport])]
    variants = list(line_df['VARIANTE'].unique())
    transport_data = dict()
    for variant in variants:        
        lines_v = lines.loc[lines['VARIANTE'].isin([variant])]
        stops_v = stops.loc[stops['Variante'].isin([variant])]
        
        v_points = list(lines_v['geometry'])[0]
        stops_data_v = dict(zip(stops_v.stop_id, stops_v.geometry))
        
        transport_data[int(variant)] = calculate_distance_between_stops(stops_data=stops_data_v, points=v_points)
        
    return transport_data

##### Get distances between the stops for all buses, trams, metros

In [6]:
transports = sorted(set(line_df['LIGNE']))
transport_distances = dict()
for transport in transports:
    transport_distances[transport] = get_stop_distances_for_transport(transport=transport, line_df=line_df, stops_df=stops_df)

##### Save the distances between the stops for all buses, trams, metros

In [8]:
with open(output_file, 'w') as f:
    dump(transport_distances, f)